# Model Driven Software Engineering for Data Warehousing — Part 1: Code Generation

> Everything should be made as simple as possible, but not simpler.
> 
> — Attributed to Albert Einstein

This article is the first part of a series of articles in which I want to give an overview of how I think Model Driven Software Engineering (MDSE) can be used for data warehousing. 
This article is about the concepts of model driven software engineering and why you should use it while developing your Data Warehouse (DWH).
Last but not least the topics described in this article are explained using an example.

## What is Model Driven (Software) Engineering?

According to Wikipedia (https://en.wikipedia.org/wiki/Model-driven_engineering, 18 November 2022):

> **Model-driven engineering** (**MDE**) is a software development methodology that focuses on creating and exploiting domain models, which are conceptual models of all the topics related to a specific problem.
>  Hence, it highlights and aims at abstract representations of the knowledge and activities that govern a particular application domain, rather than the computing (i.e. algorithmic) concepts.

In my view, model driven software engineering can be expressed in terms of (meta-)models, model-to-model transformations and code generation.

*Models* are used to describe objects in a (problem) domain.
Objects in the model should capture all important concepts of the domain.
To make sure models do not become too complex, care must be taken to not over do it: models should not contain to much details.
When I apply MDSE I use what is generally called *meta-model* (model describing a model) to descirbe what objects can be used models.
The meta-model must be complete in the sense that it captures all important concepts of the domain, but not to extensive as this tends to complicate matters.

The concepts of model and meta-model translate directly to databases.
One could say that the table structures are meta-models, they define what can be stored, for example, an email address is a string of at most 128 characters.
Primary key constraints and foreign key constraints are also part of the meta-model as they limit the models that are allowed.
The data in the tables relates to models, it contains actual data such as the email address `example@blog.com`.

What you often see in Enterprise Architecture tools, is that people develop logical data models.
Eventually, these logical models must be implemented in a database, but to do this a number of extra step are required to transform objects in the logical level to objects in the physical level.
These tools support automation of this process.
This is what is generally called *model to model transformations* or *M2M*.
For example, entities in the logical domain are transformed into tables in the physical domain, attributes are transformed into columns and relations are transformed into foreign key constraints.
Often, the model in the source domain of the transformation lacks information to fully transform into the target domain.
An example of this is that the type information of logical attributes is missing or not explit enough to directly translate into column data types in the physical domain.

![Model-to-model transformation](./img/M2M=300x.svg)

I use *templates* to generate text based on models.
What this text is depends on my requirements. It can be executable code, such as ETL statements to load data into tables, documentation about data lineage, or anything else that can be represented in a textual format.
A *template engine* is used to apply the template to the model and generating the textual result.
What a template engine basically does is that it tries to replace parts of the provided template with parts of the provided model resulting in one or more textual outputs.
This approach is referred to as *code generation*. The last section of this article contains an example that makes this more concrete.


![Code generation](./img/M2T=300x.svg)

## When to use Model Driven Software Engineering

In my experience, the practical benefits of this methodology are threefold.

1. *Reduce repetitive work*

   I believe that it is important to work of ‘fun’ things. Doing the same thing over and over again does not qualify as being fun (at least for me).
   When developing a data warehouse, many tasks can be repetitive.
   Maybe the most common example is creating a staging layer in which data from source systems is stored prior to be stored in the actual DWH.
   Often the structure of the source tables are copied and several columns to store processing data are added. 
   This activity has to be done for *every* relevant table in *every* source system. 
   I already get bored writing about it!

2. *Reduce the risk of errors*

   When doing manual work there is a high risk of introducing errors.
   Typos for instance. Often, these finding and fixing these errors requires a lot of work (the later in the development process an error is found, the more it costs to fix it).
   When applying MDSE, all text that is generated is based on the same models and templates.
   This means that is enough to validate the correctness of your models and your templates to make sure that the result of the code generation process will be correct.
   Especially when applied at large scale this will save you a lot of headaches.

3. *Increase efficiency*

   One could argue that manual work is s-l-o-w.
   According to the Guinness World Records, since 2005 the official holder of the title of fastest typer in the world on a computer is Barbara Blackburn with a top typing speed of 212 words per minute.
   Although this is way, way faster than I ever type, it is far from the performance that you get when generating your code.
   When applying MDSE at large scale, the time it takes to develop and maintain your models and templates does not compare to the time it would take to write your code manually.
   This means that developers have more time to work on fun things (or to drink coffee!).

The points in the list above indicate that it becomes profitable to apply MDSE when you have workloads that are highly repetative.
As the setup for MDSE, i.e. defining your (meta-)models, transformations and templates takes time, doing it only for a small number of applications is not efficient.

## Code Generation Example

### The Logical Domain

![Model-to-model transformation](./img/M2M-01=300x.svg)

The first step is to create the model that we need.
The model contains the `order` domain, that in turn contains three entities, `Customer`, `Product` and `Order`.
Every entity has *properties* that define the state of an etity and *relations* to other entities.

In [1]:
orders_domain_model = {
    "domains": [
        {
            "name": "orders",
            "entities": [
                {
                    "name": "Customer",
                    "properties": [
                        {
                            "name": "address",
                            "type": "string",
                        },
                    ],
                },
                {
                    "name": "Product",
                    "properties": [
                        {
                            "name": "description",
                            "type": "string",
                        },
                    ],
                },
                {
                    "name": "Order",
                    "properties": [                        
                        {
                            "name": "orderTimestamp_utc",
                            "type": "timestamp",
                        },
                        {
                            "name": "amount",
                            "type": "int",
                        },
                    ],
                    "relations": [
                        {
                            "role": "orderedBy",
                            "domain": "orders",
                            "entity": "Customer",
                        },
                        {
                            "role": "orderFor",
                            "domain": "orders",
                            "entity": "Product",
                        },
                    ],
                },
            ],
        },
    ],
}

### Transforming from the Logical Domain to the Physical Domain

![Model-to-model transformation](./img/M2M-02=300x.svg)

Now, assume that we want to create a data model in a database to store data for this domain.
To simplify this, we can to transform the model above into model that is fit to generate DML code from.
This means that we need to:

- convert domains to schemas,
- convert entities to tables,
- convert properties to columns with correct data types,
- add primary key constraints by introducing an extra `id` column to each table, and
- add foreign key constraints for each relation

To do this, we can define a *model to model transformation* (M2M).
The Python function `transform_domain_model_to_database_model(domain_model)` below implements this transformation.

In [2]:
def transform_property_datatype_to_database_datatype(property: dict) -> str:
    """
    Returns a database datatype based on the data type of the provided property.
    If the type can not be resolved, the original datatype is retunred.

    Paramters``
    ---------
    - property: 
      An object with a key `type`.
    
    Returns
    -------
    A datatype fit for a database column if applicable. Otherwise the orignial datatype.
    """
    property_datatype = property["type"]
    if "string" == property_datatype:
        return 'nvarchar(255)'
    if "timestamp" == property_datatype:
        return 'datetime2'
    return property_datatype

def transform_domain_model_to_database_model(domain_model: dict) -> dict:
    """
    Creates and transforms domain model to database model.

    Parameters
    ----------
    - domain_model:
      An object containing domain model.

    Returns
    -------
    An object containing database modela.
    """
    schemas = []
    database_model = {
        "schemas": schemas
    }

    for domain in domain_model["domains"]:
        # Transform a domain into a database schema.
        schema_tables = []
        schemas.append({
            "name": domain["name"],
            "tables": schema_tables
        })

        for entity in domain["entities"]:
            # Transoform an entity into a table.
            table_foreign_key_constraints = []
            # Every table gets an extra 'id' column. 
            table_columns = [{
                "name": "id",
                "type": "int",
                "nullable": False
            }]
            table_model = {
                "name": entity["name"],
                "columns": table_columns,
                # The 'id' column of each table is its primary key.
                "primary_key_constraint": {
                    "name": f"{entity['name']}_pkey",
                    "column_names": [
                        "id"
                    ],
                },
                "foreign_key_constraints": table_foreign_key_constraints
            }
            schema_tables.append(table_model)

            for property in entity.get("properties", []):
                # Transform a property into a column
                table_columns.append({
                    "name": property["name"],
                    "type": transform_property_datatype_to_database_datatype(property),
                    "nullable": True
                })

            for relation in entity.get("relations", []):
                # Transform a relation into a foreign key constraint.
                # To be able to create a foreign key constraint an extra columns is added to the table.
                table_columns.append({
                    "name": f"{relation['role']}_{relation['entity']}_id",
                    "type": "int",
                    "nullable": False
                })

                # Transform a relation into a foreign key constraint
                table_foreign_key_constraints.append({
                    "name": f"{entity['name']}_{relation['role']}_{relation['entity']}",
                    "to_schema_name": relation['domain'],
                    "to_table_name": relation['entity'],
                    # Foreign keys are defined between the added attribute and the primary key attribute of the referenced table.
                    "from_column_names": [
                        f"{relation['role']}_{relation['entity']}_id"
                    ],
                    "to_column_names": [
                        "id"
                    ]
                })

    return database_model

orders_database_model = transform_domain_model_to_database_model(orders_domain_model)

### Code Generation

![Model-to-model transformation](./img/M2T-02=300x.svg)

Now the database model is created, we can easily generate DDL code.
The code below shows a simple Jinja2 template that is used to generate the code.
There are a number of important things to notice.

- `{% for value in sequence %} ... {% endfor %}`
- `{{ expression }}`

For example, the following template

```
{% for name in ['World', 'Maarten', 'Reader'] %}
Hello {{ name }}!
{% endfor %}
```

results in the following output.

```
Hello World!

Hello Maarten!

Hello Reader!
```

In [3]:
ddl_template_text = """
{% for schema in schemas %}
GO
CREATE SCHEMA [{{ schema.name }}];
GO

{% for table in schema.tables %}
CREATE TABLE [{{ schema.name }}].[{{ table.name }}] (
{% for column in table.columns -%}
{{ '  ' if loop.index == 1 else ', ' }}[{{ column.name }}] {{ column.type }} {{ 'NULL' if column.nullable else 'NOT NULL' }}
{% endfor -%}
, CONSTRAINT [{{ table.primary_key_constraint.name }}] PRIMARY KEY ([{{ table.primary_key_constraint.column_names|join('], [') }}])
);
{% endfor %}

{% for table in schema.tables|selectattr('foreign_key_constraints') %}
ALTER TABLE [{{ schema.name }}].[{{ table.name }}]
ADD
{% for foreign_key in table.foreign_key_constraints -%}
{{ '  ' if loop.index == 1 else ', ' }}CONSTRAINT [{{ foreign_key.name }}] FOREIGN KEY ([{{ foreign_key.from_column_names|join('], [') }}]) REFERENCES [{{ foreign_key.to_schema_name }}].[{{ foreign_key.to_table_name }}] ([{{ foreign_key.to_column_names|join('], [') }}])
{% endfor -%}
;
{% endfor %}

{% endfor %}
"""

def generate_code(template_text: str, model: dict) -> str: 
    """
    Generates code based on a model dictionary and a template string.

    Parameters
    ----------

    Returns
    -------
    """
    from jinja2 import Environment, BaseLoader
    template = Environment(loader=BaseLoader).from_string(template_text)

    return template.render(**model).strip() + '\n'

def generate_ddl_code(database_model: dict) -> str:
    """
    Generates DDL code based on database model.

    Parameters
    ----------
    - database_model
      The model used to generate the DDL code.

    Returns
    -------
    The generated DDL code.
    """
    return generate_code(ddl_template_text, database_model)

%reload_ext autoreload
%autoreload 2
import project_path
from src.article_utils import visualzize_sql_code
visualzize_sql_code(generate_ddl_code(orders_database_model))

```sql
GO
CREATE SCHEMA [orders];
GO


CREATE TABLE [orders].[Customer] (
  [id] int NOT NULL
, [address] nvarchar(255) NULL
, CONSTRAINT [Customer_pkey] PRIMARY KEY ([id])
);

CREATE TABLE [orders].[Product] (
  [id] int NOT NULL
, [description] nvarchar(255) NULL
, CONSTRAINT [Product_pkey] PRIMARY KEY ([id])
);

CREATE TABLE [orders].[Order] (
  [id] int NOT NULL
, [orderTimestamp_utc] datetime2 NULL
, [amount] int NULL
, [orderedBy_Customer_id] int NOT NULL
, [orderFor_Product_id] int NOT NULL
, CONSTRAINT [Order_pkey] PRIMARY KEY ([id])
);



ALTER TABLE [orders].[Order]
ADD
  CONSTRAINT [Order_orderedBy_Customer] FOREIGN KEY ([orderedBy_Customer_id]) REFERENCES [orders].[Customer] ([id])
, CONSTRAINT [Order_orderFor_Product] FOREIGN KEY ([orderFor_Product_id]) REFERENCES [orders].[Product] ([id])
;

```

Now, we also want to start to work on our actual data warehouse.
At Nippur, the starting point for that is to create historical storage of our data.
We call this the *Historical Data Archive* (HDA).

To do this, first make sure we can actually store the data in HDA table we transform our previous database model into HDA database model and generated DDL code.

In [5]:
from typing import Tuple
def transform_database_model_into_hda_model_and_mappings(database_model: dict) -> Tuple[dict, dict]:
    """
    Extends existing database model to HDA model.
    Also creates model that describes the column mappings between the database and the HDA.

    Parameters
    ----------
    - database_model
      the model that needs to be transformed.

    Returns
    -------
    A tuple (hda_model, mappings_model), where
    - hda_model contains database model for the HDA and ,
    - mappings_model model describing the mappings between the provided database model and the HDA model.
    """
    hda_schemas = []
    hda_model = {
        "schemas": hda_schemas
    }

    schema_mappings = []
    hda_mappings = {
        "schema_mappings": schema_mappings
    }

    for schema in database_model["schemas"]:
        hda_tables = []
        hda_schemas.append({
            "name": f"{schema['name']}_hda",
            "tables": hda_tables
        })

        table_mappings = []
        schema_mappings.append({
            "table_mappings": table_mappings
        })

        for table in schema["tables"]: 
            hda_columns = [{
                "name": "hda_registration_utc",
                "type": "datetime2"
            }] + table["columns"]

            hda_tables.append({
                "name": table["name"],
                "columns": hda_columns,
                "primary_key_constraint": {
                    "name": f"{table['name']}_pkey",
                    "column_names": table["primary_key_constraint"]["column_names"] + ["hda_registration_utc"],
                }
            })

            column_mappings = [{
                "expression": "GETUTCDATE()",
                "target": "hda_registration_utc"
            }]
            table_mappings.append({
                "source": {
                    "schema": schema["name"],
                    "table": table["name"]
                },
                "target": {
                    "schema": f"{schema['name']}_hda",
                    "table": table["name"]
                },
                "column_mappings": column_mappings
            })

            for column in table["columns"]: 
                column_mappings.append({
                    "source": column["name"],
                    "target": column["name"]
                })

    return hda_model, hda_mappings

etl_template_text = """
{% for schema_mapping in schema_mappings %}
{% for table_mapping in schema_mapping.table_mappings %}
INSERT INTO [{{ table_mapping.target.schema }}].[{{ table_mapping.target.table }}] (
{% for column_mapping in table_mapping.column_mappings -%}    
{{ '  ' if loop.index == 1 else ', ' }}[{{ column_mapping.target }}]
{% endfor -%}
)
SELECT
{% for column_mapping in table_mapping.column_mappings -%}   
{{ '  ' if loop.index == 1 else ', ' }}{% if 'expression' in column_mapping %}{{ column_mapping.expression }}{% elif 'source' in column_mapping %}[{{ column_mapping.source }}]{% else %}NULL{% endif %}
{% endfor -%}
FROM
  [{{ table_mapping.source.schema }}].[{{ table_mapping.source.table }}]
;
{% endfor -%}
{% endfor %}
"""

pit_template_text = """
{% for schema in schemas %}
GO
CREATE SCHEMA [{{ schema.name[:-4] }}_pub]
GO

{% for table in schema.tables %}
CREATE OR ALTER FUNCTION [{{ schema.name[:-4] }}_pub].[{{ table.name }}_pit] (
  @timestamp_utc datetime2
)
RETURNS TABLE
AS
  RETURN
    SELECT
    {% for column in table.columns -%}
    {{ '  ' if loop.index == 1 else ', ' }}[{{ column.name }}]
    {% endfor -%}
    FROM
      [{{ schema.name }}].[{{ table.name }}] [t1]
    WHERE
      [hda_registration_utc] < @timestamp_utc
    AND NOT EXISTS (
      SELECT 
        NULL
      FROM
        [{{ schema.name }}].[{{ table.name }}] [t2]
      WHERE
        t2.[hda_registration_utc] > t1.[hda_registration_utc]
    )
;
GO
{% endfor %}

{% endfor %}
"""

def generate_etl_code(mapping_model: dict) -> str:
    return generate_code(etl_template_text, mapping_model)
    
def generate_pit_code(hda_model: dict) -> str:
    return generate_code(pit_template_text, hda_model)

hda_database_model, hda_mapping_model = transform_database_model_into_hda_model_and_mappings(orders_database_model)
visualzize_sql_code(generate_ddl_code(hda_database_model))
visualzize_sql_code(generate_pit_code(hda_database_model))
visualzize_sql_code(generate_etl_code(hda_mapping_model))

```sql
GO
CREATE SCHEMA [orders_hda];
GO


CREATE TABLE [orders_hda].[Customer] (
  [hda_registration_utc] datetime2 NOT NULL
, [id] int NOT NULL
, [address] nvarchar(255) NULL
, CONSTRAINT [Customer_pkey] PRIMARY KEY ([id], [hda_registration_utc])
);

CREATE TABLE [orders_hda].[Product] (
  [hda_registration_utc] datetime2 NOT NULL
, [id] int NOT NULL
, [description] nvarchar(255) NULL
, CONSTRAINT [Product_pkey] PRIMARY KEY ([id], [hda_registration_utc])
);

CREATE TABLE [orders_hda].[Order] (
  [hda_registration_utc] datetime2 NOT NULL
, [id] int NOT NULL
, [orderTimestamp_utc] datetime2 NULL
, [amount] int NULL
, [orderedBy_Customer_id] int NOT NULL
, [orderFor_Product_id] int NOT NULL
, CONSTRAINT [Order_pkey] PRIMARY KEY ([id], [hda_registration_utc])
);

```

```sql
GO
CREATE SCHEMA [orders_pub]
GO


CREATE OR ALTER FUNCTION [orders_pub].[Customer_pit] (
  @timestamp_utc datetime2
)
RETURNS TABLE
AS
  RETURN
    SELECT
      [hda_registration_utc]
    , [id]
    , [address]
    FROM
      [orders_hda].[Customer] [t1]
    WHERE
      [hda_registration_utc] < @timestamp_utc
    AND NOT EXISTS (
      SELECT 
        NULL
      FROM
        [orders_hda].[Customer] [t2]
      WHERE
        t2.[hda_registration_utc] > t1.[hda_registration_utc]
    )
;
GO

CREATE OR ALTER FUNCTION [orders_pub].[Product_pit] (
  @timestamp_utc datetime2
)
RETURNS TABLE
AS
  RETURN
    SELECT
      [hda_registration_utc]
    , [id]
    , [description]
    FROM
      [orders_hda].[Product] [t1]
    WHERE
      [hda_registration_utc] < @timestamp_utc
    AND NOT EXISTS (
      SELECT 
        NULL
      FROM
        [orders_hda].[Product] [t2]
      WHERE
        t2.[hda_registration_utc] > t1.[hda_registration_utc]
    )
;
GO

CREATE OR ALTER FUNCTION [orders_pub].[Order_pit] (
  @timestamp_utc datetime2
)
RETURNS TABLE
AS
  RETURN
    SELECT
      [hda_registration_utc]
    , [id]
    , [orderTimestamp_utc]
    , [amount]
    , [orderedBy_Customer_id]
    , [orderFor_Product_id]
    FROM
      [orders_hda].[Order] [t1]
    WHERE
      [hda_registration_utc] < @timestamp_utc
    AND NOT EXISTS (
      SELECT 
        NULL
      FROM
        [orders_hda].[Order] [t2]
      WHERE
        t2.[hda_registration_utc] > t1.[hda_registration_utc]
    )
;
GO

```

```sql
INSERT INTO [orders_hda].[Customer] (
  [hda_registration_utc]
, [id]
, [address]
)
SELECT
  GETUTCDATE()
, [id]
, [address]
FROM
  [orders].[Customer]
;

INSERT INTO [orders_hda].[Product] (
  [hda_registration_utc]
, [id]
, [description]
)
SELECT
  GETUTCDATE()
, [id]
, [description]
FROM
  [orders].[Product]
;

INSERT INTO [orders_hda].[Order] (
  [hda_registration_utc]
, [id]
, [orderTimestamp_utc]
, [amount]
, [orderedBy_Customer_id]
, [orderFor_Product_id]
)
SELECT
  GETUTCDATE()
, [id]
, [orderTimestamp_utc]
, [amount]
, [orderedBy_Customer_id]
, [orderFor_Product_id]
FROM
  [orders].[Order]
;

```

In [6]:
markdown_template_text = """
# Lineage Documentation

{% for schema_mapping in schema_mappings %}
{% for table_mapping in schema_mapping.table_mappings %}
## [{{ table_mapping.target.schema }}].[{{ table_mapping.target.table }}]

| target column | source |
| :----- | :----- | 
{%- for column_mapping in table_mapping.column_mappings %}
| {{ column_mapping.target }} | {% if 'expression' in column_mapping %}`{{ column_mapping.expression }}`{% elif 'source' in column_mapping %}[{{ table_mapping.source.schema }}].[{{table_mapping.source.table  }}].[{{ column_mapping.source }}]{% else %}`NULL`{% endif %} |
{%- endfor -%}
{% endfor %}
{% endfor %}
"""

def generate_lineage_code(mapping_model: dict) -> str:
    return generate_code(markdown_template_text, mapping_model)

from src.article_utils import visualzize_markdown_code
visualzize_markdown_code(generate_lineage_code(hda_mapping_model))

# Lineage Documentation



## [orders_hda].[Customer]

| target column | source |
| :----- | :----- |
| hda_registration_utc | `GETUTCDATE()` |
| id | [orders].[Customer].[id] |
| address | [orders].[Customer].[address] |
## [orders_hda].[Product]

| target column | source |
| :----- | :----- |
| hda_registration_utc | `GETUTCDATE()` |
| id | [orders].[Product].[id] |
| description | [orders].[Product].[description] |
## [orders_hda].[Order]

| target column | source |
| :----- | :----- |
| hda_registration_utc | `GETUTCDATE()` |
| id | [orders].[Order].[id] |
| orderTimestamp_utc | [orders].[Order].[orderTimestamp_utc] |
| amount | [orders].[Order].[amount] |
| orderedBy_Customer_id | [orders].[Order].[orderedBy_Customer_id] |
| orderFor_Product_id | [orders].[Order].[orderFor_Product_id] |
